<a href="https://colab.research.google.com/github/ankitojha1999/Autogluon_Part1/blob/main/Ieee_fraud_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('mount')

!mkdir -p ~/.kaggle
!cp /content/mount/MyDrive/kaggle.json ~/.kaggle/kaggle.json


Drive already mounted at mount; to attempt to forcibly remount, call drive.mount("mount", force_remount=True).


In [3]:
!pip install kaggle

In [5]:
!rm -rf /content/ieee-fraud-detection
!mkdir -p /content/ieee-fraud-detection
!kaggle competitions download -c ieee-fraud-detection -p /content/ieee-fraud-detection

!unzip /content/ieee-fraud-detection/ieee-fraud-detection.zip -d /content/ieee-fraud-detection/

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.10/dist-packages/kaggle/__init__.py", line 7, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 414, in authenticate
    self._load_config(config_data)
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 465, in _load_config
    raise ValueError('Error: Missing %s in configuration.' % item)
ValueError: Error: Missing username in configuration.
unzip:  cannot find or open /content/ieee-fraud-detection/ieee-fraud-detection.zip, /content/ieee-fraud-detection/ieee-fraud-detection.zip.zip or /content/ieee-fraud-detection/ieee-fraud-detection.zip.ZIP.


In [6]:
!pip install autogluon

In [15]:
import pandas as pd
import numpy as np
from autogluon.tabular import TabularPredictor

directory = '/content/ieee-fraud-detection/'
label = 'isFraud'
eval_metric = 'roc_auc'
save_path = directory + 'auto-gluon-model'

train_identity = pd.read_csv(directory + 'train_identity.csv')
train_transaction = pd.read_csv(directory + 'train_transaction.csv')
print(f"train_transaction columns: {train_transaction.columns.tolist()}")
print(f"train_identity columns: {train_identity.columns.tolist()}")


train_transaction columns: ['TransactionID', 'isFraud', 'TransactionDT', 'TransactionAmt', 'ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5', 'card6', 'addr1', 'addr2', 'dist1', 'dist2', 'P_emaildomain', 'R_emaildomain', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15', 'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'V29', 'V30', 'V31', 'V32', 'V33', 'V34', 'V35', 'V36', 'V37', 'V38', 'V39', 'V40', 'V41', 'V42', 'V43', 'V44', 'V45', 'V46', 'V47', 'V48', 'V49', 'V50', 'V51', 'V52', 'V53', 'V54', 'V55', 'V56', 'V57', 'V58', 'V59', 'V60', 'V61', 'V62', 'V63', 'V64', 'V65', 'V66', 'V67', 'V68', 'V69', 'V70', 'V71', 'V72', 'V73', 'V74', 'V75', 'V76', 'V77', 'V7

In [18]:
train_data = pd.merge(train_transaction, train_identity, on='TransactionID', how='left')

In [19]:
predictor = TabularPredictor(label=label, eval_metric=eval_metric, path=save_path, verbosity=2).fit(
    train_data.sample(n=500), presets='high_quality',time_limit=3600,num_gpus=1)

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
CPU Count:          2
Memory Avail:       3.85 GB / 12.67 GB (30.4%)
Disk Space Avail:   61.10 GB / 107.72 GB (56.7%)
Presets specified: ['high_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Note: `save_bag_folds=False`! This will greatly reduce peak disk usage during fit (by ~8x), but runs the risk of an out-of-memory error during model refit if memory is small relative to the data size.
	You can avoid this risk by setting `save_bag_folds=True`.
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input da

In [20]:
results = predictor.fit_summary()

*** Summary of fit() ***
Estimated performance of each model:
                             model  score_val eval_metric  pred_time_val     fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0              WeightedEnsemble_L3   0.895670     roc_auc      36.352790  1933.032012                0.000688           0.220749            3      False         49
1              WeightedEnsemble_L2   0.844674     roc_auc       8.560995   212.246028                0.001003           0.313127            2      False         34
2        NeuralNetTorch_r79_BAG_L2   0.766323     roc_auc      31.312760  1637.746307                3.376046          83.640799            2      False         46
3                LightGBMXT_BAG_L2   0.732027     roc_auc      28.171375  1582.971227                0.234661          28.865719            2      False         35
4             LightGBMLarge_BAG_L2   0.712096     roc_auc      28.251184  1602.114800                0.314470         